<a href="https://colab.research.google.com/github/Fariha-Asif/voice-to-voice-chatbot/blob/main/voicetovoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install git+https://github.com/openai/whisper.git
!pip install groq gtts gradio


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-hkhu4909
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-hkhu4909
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [15]:
import os
import whisper  # Import whisper for local transcription
from groq import Groq
from gtts import gTTS
import gradio as gr
import tempfile

# Load the whisper model
whisper_model = whisper.load_model("base")  # You can also use "small", "medium", "large" based on your preference

# Set your API key for Groq
os.environ['GROQ_API_KEY'] = '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'

# Function to transcribe audio using the local Whisper model
def transcribe_audio(audio_file):
    try:
        print(f"Transcribing audio using Whisper: {audio_file}")  # Debugging
        # Perform transcription using the whisper model
        result = whisper_model.transcribe(audio_file)
        print(f"Whisper transcription result: {result['text']}")  # Debugging
        return result['text']
    except Exception as e:
        print(f"Error during transcription: {e}")
        return "Error during transcription"

# Function to interact with Groq's language model
def chat_with_groq(user_input):
    try:
        client = Groq(api_key=os.getenv('GROQ_API_KEY'))
        print(f"Sending input to Groq LLM: {user_input}")  # Debugging print
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": user_input}],
            model="llama3-8b-8192"
        )
        print(f"Groq LLM response: {response}")  # Debugging
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error interacting with Groq LLM: {e}")
        return "Error interacting with LLM"

# Function to convert text to speech using gTTS
def text_to_speech(text):
    try:
        print(f"Converting text to speech: {text}")  # Debugging
        tts = gTTS(text=text, lang='en')
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tts.save(temp_file.name)
        print(f"Saved audio response: {temp_file.name}")  # Debugging
        return temp_file.name
    except Exception as e:
        print(f"Error during text-to-speech conversion: {e}")
        return None

# Main function to process audio input, get LLM response, and return audio output
def process_audio(audio):
    try:
        # Step 1: Transcribe the audio input
        transcription = transcribe_audio(audio)
        if transcription.startswith("Error"):
            return None

        # Step 2: Get the chatbot response from Groq's LLM
        response = chat_with_groq(transcription)
        if response.startswith("Error"):
            return None

        # Step 3: Convert the chatbot response to speech
        audio_response = text_to_speech(response)
        if not audio_response:
            return None

        return audio_response
    except Exception as e:
        print(f"Error in process_audio: {e}")
        return None

# Gradio interface setup
interface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath"),  # Using 'filepath' for file handling
    outputs=gr.Audio(type="filepath"), # Using 'filepath' for file handling
    title="Real-time Voice-to-Voice Chatbot",
    description="Speak to the bot, and it will respond with a voice message!"
)

# Launch the Gradio app
interface.launch()


100%|███████████████████████████████████████| 139M/139M [00:07<00:00, 18.9MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://915f69036ed1d62579.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
